# TrinityStrategy Indicator Analysis Dashboard

This notebook provides **automated** comprehensive analysis and visualization for **all instruments** configured in `uout.json`.

## Features:
- **Automated Multi-Instrument Processing**: Reads markets/securities from `uout.json` and processes all instruments
- **Pattern 2 Connection Reuse**: Efficient single-connection approach for fetching multiple instruments
- **Organized Output Structure**: Charts saved to `tier-1_output/charts/<instrument>/`
- **Three Chart Types Per Instrument**:
  - **Scout Time Series**: ADX, DI+, DI-, Bollinger Bands, Conviction Oscillator
  - **Distribution Analysis**: Histograms of all Scout indicator values
  - **Correlation Matrix**: Relationships between Scout indicators

## Scout Indicators Visualized:
- **TrendScout**: ADX, DI+, DI- (regime detection - trending vs ranging)
- **TensionScout**: Bollinger Bands (volatility and overbought/oversold)
- **CrowdScout**: Conviction Oscillator (volume confirmation)

## Usage:
1. Update `TOKEN`, `START_DATE`, and `END_DATE` in the configuration cell
2. Run all cells sequentially
3. Charts will be automatically generated for all instruments in `tier-1_output/charts/`

**Note**: No need to specify individual markets/commodities - they're auto-parsed from `uout.json`!

In [ ]:
# Core imports
import asyncio
import warnings
from typing import Dict, List

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import seaborn as sns
import svr3

warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

print('📊 TrinityStrategy Indicator Dashboard Initialized')

In [ ]:
# Additional imports for automation
import json
import os

def parse_uout_instruments():
    """Parse uout.json to get all (market, security) pairs for visualization
    
    Returns:
        List of tuples: [(market, code), ...] e.g., [('DCE', 'i<00>'), ('SHFE', 'cu<00>'), ...]
    """
    with open('uout.json', 'r') as f:
        config = json.load(f)
    
    # Access nested structure: uout.json has "private" as top-level key
    private_config = config['private']
    
    markets = private_config['markets']  # e.g., ['DCE', 'SHFE']
    securities_per_market = private_config['securities']  # e.g., [['i'], ['cu', 'sc']]
    
    # Build instrument list: [(market, code<00>), ...]
    instruments = []
    for market, securities in zip(markets, securities_per_market):
        for security in securities:
            # Append logical contract code (e.g., 'i<00>')
            instruments.append((market, f'{security}<00>'))
    
    return instruments

print('✓ Config parser and automation utilities loaded')

## Configuration

Update these settings to match your test data:

In [ ]:
# Server configuration
RAILS_URL = 'https://10.99.100.116:4433/private-api/'
WS_URL = 'wss://10.99.100.116:4433/tm'
TM_MASTER = ('10.99.100.116', 6102)

TOKEN = '58abd12edbde042536637bfba9d20d5faf366ef481651cdbb046b1c3b4f7bf7a97ae7a2e6e5dc8fe05cd91147c8906f8a82aaa1bb1356d8cb3d6a076eadf5b5a' 

# Date range matching your quick test (7 days: Oct 25 - Nov 1, 2024)
START_DATE = 20240101000000
END_DATE = 20250101000000

# Indicator configuration
INDICATOR_NAME = 'TrinityStrategy'
GRANULARITY = 900  # 15-minute bars
NAMESPACE = 'private'

# Output directory for organized charts
OUTPUT_DIR = 'tier-1_output/charts'

# NOTE: Market/commodity pairs are now auto-parsed from uout.json
# No need to manually specify MARKET and COMMODITY variables

print(f'Configuration: {INDICATOR_NAME} from {START_DATE} to {END_DATE}')
print(f'Output directory: {OUTPUT_DIR}')

## Data Fetcher Class

Handles server connection and data retrieval using svr3.

In [ ]:
class TrinityStrategyDataFetcher:
    """Data fetcher for TrinityStrategy indicator using svr3 module"""

    def __init__(self, token: str, start: int, end: int):
        self.token = token
        self.start_date = start
        self.end_date = end
        self.client = None
        self.df = None
        self.available_fields = []

    async def connect(self):
        """Establish connection to server (Pattern 2: will update markets/codes per fetch)"""
        print(f'🔄 Connecting to server...')

        # Initialize with dummy market/code (will be updated in fetch())
        self.client = svr3.sv_reader(
            self.start_date, self.end_date,
            INDICATOR_NAME, GRANULARITY, NAMESPACE,
            'symbol', ['DCE'], ['i<00>'],  # Dummy initial values
            False, RAILS_URL, WS_URL,
            '', '', TM_MASTER,
        )
        self.client.token = self.token

        await self.client.login()
        await self.client.connect()
        self.client.ws_task = asyncio.create_task(self.client.ws_loop())
        await self.client.shakehand()

        print(f'✓ Connected to server')

    async def fetch(self, market: str, code: str) -> pd.DataFrame:
        """Fetch indicator data for specified market/code (reuses connection)"""
        print(f'📊 Fetching indicators for {market}/{code}...')

        # Update markets/codes for this fetch (Pattern 2 from wos/10-visualization.md)
        self.client.markets = [market]
        self.client.codes = [code]
        self.client.namespace = NAMESPACE  # private namespace

        ret = await self.client.save_by_symbol()
        data = ret[1][1]

        if not data:
            print(f'⚠ No indicator data returned for {market}/{code}')
            return pd.DataFrame()

        df = pd.DataFrame(data)

        header_fields = ['time_tag', 'granularity', 'market', 'code', 'namespace']
        self.available_fields = [col for col in df.columns if col not in header_fields]

        if 'time_tag' in df.columns:
            df['datetime'] = pd.to_datetime(df['time_tag'], unit='ms')
            df = df.sort_values('datetime')

        self.df = df

        print(f'✓ Loaded {len(df)} indicator data points')
        if 'datetime' in df.columns:
            print(f'  Date range: {df["datetime"].min()} to {df["datetime"].max()}')
        print(f'  Available fields: {", ".join(self.available_fields)}')

        return df

    async def close(self):
        """Clean up connection"""
        if self.client:
            self.client.stop()
            await self.client.join()
            print('✓ Connection closed')

    def get_summary(self) -> Dict:
        """Get summary statistics"""
        if self.df is None or self.df.empty:
            return {}

        summary = {
            'total_points': len(self.df),
            'fields': self.available_fields,
        }

        if 'datetime' in self.df.columns:
            summary['date_range'] = (self.df['datetime'].min(), self.df['datetime'].max())

        numeric_fields = self.df.select_dtypes(include=[np.number]).columns
        summary['statistics'] = {}
        for field in numeric_fields:
            if field not in ['time_tag', 'bar_index']:
                summary['statistics'][field] = {
                    'mean': self.df[field].mean(),
                    'std': self.df[field].std(),
                    'min': self.df[field].min(),
                    'max': self.df[field].max()
                }

        return summary


async def fetch_price_data_standalone(token: str, start: int, end: int, market: str, code: str) -> pd.DataFrame:
    """Fetch OHLCV price data (SampleQuote) with separate dedicated connection
    
    This function creates its own svr3.sv_reader instance specifically for fetching
    SampleQuote dependency data from the global namespace. Cannot reuse the indicator
    connection because algo_name is baked into the connection initialization.
    
    Pattern from: IronOreIndicator/analysis.ipynb (working example)
    """
    print(f'💹 Fetching price data for {market}/{code}...')
    
    try:
        # Create separate reader for SampleQuote
        reader = svr3.sv_reader(
            start, end,
            'SampleQuote',      # ✅ Different algo from indicator
            GRANULARITY,
            'global',           # ✅ Global namespace for dependencies
            'symbol',
            [market],
            [code],
            False,
            RAILS_URL, WS_URL,
            '', '',
            TM_MASTER,
        )
        reader.token = token
        
        # Connect
        await reader.login()
        await reader.connect()
        reader.ws_task = asyncio.create_task(reader.ws_loop())
        await reader.shakehand()
        
        # Fetch data
        ret = await reader.save_by_symbol()
        
        # Extract data correctly: ret[1][1] is the List[Dict]
        if ret and len(ret) > 1 and len(ret[1]) > 1:
            data = ret[1][1]
        else:
            print(f'⚠ No price data returned for {market}/{code}')
            reader.stop()
            await reader.join()
            return pd.DataFrame()
        
        # Cleanup connection
        reader.stop()
        await reader.join()
        
        if not data:
            print(f'⚠ Empty price data for {market}/{code}')
            return pd.DataFrame()
        
        # Convert to DataFrame
        df = pd.DataFrame(data)
        
        if 'time_tag' in df.columns:
            df['datetime'] = pd.to_datetime(df['time_tag'], unit='ms')
            df = df.sort_values('datetime')
        
        print(f'✓ Loaded {len(df)} price data points')
        if 'datetime' in df.columns:
            print(f'  Date range: {df["datetime"].min()} to {df["datetime"].max()}')
        
        # Check for OHLCV fields
        expected_fields = ['open', 'high', 'low', 'close', 'volume']
        available_price_fields = [f for f in expected_fields if f in df.columns]
        print(f'  Price fields: {", ".join(available_price_fields)}')
        
        return df
        
    except Exception as e:
        print(f'❌ Error fetching price data: {e}')
        import traceback
        traceback.print_exc()
        return pd.DataFrame()


print('✓ Data fetcher class and standalone price fetcher defined')

## Plotting Functions

Defines three plotting functions for generating charts:
1. **plot_trinity_scouts**: Time series chart with all three Scout indicators
2. **plot_distributions**: Distribution histograms for Scout indicator values
3. **plot_correlation_matrix**: Correlation heatmap between Scout indicators

All functions accept `output_path` and `title_suffix` parameters for automated multi-instrument use.

In [ ]:
def filter_trading_hours(df: pd.DataFrame) -> pd.DataFrame:
    """Filter dataframe to only include trading hours
    
    Chinese commodity exchange trading hours:
    - Day session: 09:00-15:00
    - Night session: 21:00-01:00 (next day)
    """
    if df.empty or 'datetime' not in df.columns:
        return df
    
    # Extract hour and minute for precise filtering
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute
    
    # Keep only trading hours:
    # Day session: 9:00-14:59 (before 15:00)
    # Night session: 21:00-23:59 and 00:00-00:59 (midnight hour only, not 01:xx)
    trading_hours_mask = (
        ((df['hour'] >= 9) & (df['hour'] < 15)) |  # Day session: 09:00-14:59
        ((df['hour'] >= 21) & (df['hour'] <= 23)) |  # Night session: 21:00-23:59
        (df['hour'] == 0)  # Midnight hour: 00:00-00:59 only
    )
    
    filtered_df = df[trading_hours_mask].copy()
    filtered_df.drop(['hour', 'minute'], axis=1, inplace=True)
    
    removed_count = len(df) - len(filtered_df)
    if removed_count > 0:
        print(f'  🔍 Filtered: {len(df)} → {len(filtered_df)} points (removed {removed_count} non-trading hours)')
    
    return filtered_df

print('✓ Trading hours filter defined')

In [ ]:
def determine_resampling_freq(df: pd.DataFrame) -> tuple:
    """Determine appropriate resampling frequency based on data length
    
    Returns:
        (resample_freq, use_candlesticks, description)
    
    Strategy:
    - <= 100 bars: No resampling, full candlesticks
    - 101-300 bars: No resampling, simplified candlesticks  
    - 301-1000 bars: Resample to daily, line charts
    - > 1000 bars: Resample to weekly, line charts
    """
    n_bars = len(df)
    date_range = (df['datetime'].max() - df['datetime'].min()).days
    
    if n_bars <= 100:
        return None, True, f'{n_bars} bars (15-min resolution)'
    elif n_bars <= 300:
        return None, True, f'{n_bars} bars (15-min resolution, simplified)'
    elif n_bars <= 1000 or date_range <= 60:
        return 'D', False, f'{n_bars} bars → Daily resolution ({date_range} days)'
    else:
        return 'W', False, f'{n_bars} bars → Weekly resolution ({date_range} days)'


def resample_ohlcv_data(df: pd.DataFrame, freq: str) -> pd.DataFrame:
    """Resample OHLCV data to specified frequency
    
    Args:
        df: DataFrame with datetime index and OHLCV columns
        freq: Pandas frequency string ('D' for daily, 'W' for weekly)
    
    Returns:
        Resampled DataFrame with OHLCV data and indicators
    """
    # Set datetime as index for resampling
    df_indexed = df.set_index('datetime')
    
    # Define aggregation rules
    agg_dict = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }
    
    # Add indicator fields with appropriate aggregation
    indicator_fields = ['adx_value', 'di_plus', 'di_minus', 
                       'upper_band', 'middle_band', 'lower_band',
                       'conviction_oscillator']
    
    for field in indicator_fields:
        if field in df_indexed.columns:
            agg_dict[field] = 'last'  # Use last value for indicators
    
    # Resample
    resampled = df_indexed.resample(freq).agg(agg_dict).dropna()
    
    # Reset index to get datetime back as column
    resampled = resampled.reset_index()
    
    print(f'  📊 Resampled: {len(df)} → {len(resampled)} bars (frequency: {freq})')
    
    return resampled


def plot_trinity_scouts(df: pd.DataFrame, output_path: str, title_suffix: str = ''):
    """Plot the 'Golden Rule' visualization: Price + Indicators
    
    3-Panel Layout:
    - Panel 1: "Action" - Candlesticks/Lines + Bollinger Bands (what's happening)
    - Panel 2: "Regime" - ADX + DI+/DI- (how to interpret it)
    - Panel 3: "Confirmation" - Conviction Oscillator (is it real?)
    
    Features adaptive rendering based on data length:
    - Short periods (<= 300 bars): Full candlesticks
    - Medium periods (301-1000 bars): Daily resampling + line charts
    - Long periods (> 1000 bars): Weekly resampling + line charts
    
    Args:
        df: DataFrame with OHLCV price data AND indicator data
        output_path: Full path where to save the chart
        title_suffix: Additional text to append to chart titles (e.g., market/code)
    """
    if df.empty or 'datetime' not in df.columns:
        print('⚠ No datetime data available for plotting')
        return
    
    # Check for required OHLCV fields
    required_ohlcv = ['open', 'high', 'low', 'close', 'volume']
    if not all(col in df.columns for col in required_ohlcv):
        print(f'⚠ Missing OHLCV fields for candlestick chart')
        return

    # Determine adaptive resampling strategy
    resample_freq, use_candlesticks, strategy_desc = determine_resampling_freq(df)
    print(f'  📈 Rendering strategy: {strategy_desc}')
    
    # Apply resampling if needed
    plot_df = df.copy()
    if resample_freq:
        plot_df = resample_ohlcv_data(plot_df, resample_freq)
    
    # Calculate adaptive figure width (minimum 18, scale up for more data)
    n_points = len(plot_df)
    fig_width = min(max(18, n_points * 0.15), 36)  # Cap at 36 inches

    # Create figure with 3 subplots
    fig = plt.figure(figsize=(fig_width, 14))
    
    # Define grid: 3 rows with different heights
    # Panel 1 (Price+BB) gets most space, Panel 2 (ADX) and Panel 3 (Conviction) smaller
    gs = fig.add_gridspec(3, 1, height_ratios=[3, 1.5, 1.5], hspace=0.3)
    ax1 = fig.add_subplot(gs[0])  # Candlesticks + Bollinger Bands
    ax2 = fig.add_subplot(gs[1])  # ADX + DI+/DI-
    ax3 = fig.add_subplot(gs[2])  # Conviction Oscillator
    
    # ============================================================================
    # PANEL 1: "ACTION" - Price + Bollinger Bands
    # ============================================================================
    
    if use_candlesticks:
        # Plot candlesticks manually for short periods
        price_data = plot_df[['datetime', 'open', 'high', 'low', 'close']].copy()
        price_data = price_data.set_index('datetime')
        
        for idx in range(len(price_data)):
            row = price_data.iloc[idx]
            x = idx
            open_price = row['open']
            close_price = row['close']
            high_price = row['high']
            low_price = row['low']
            
            # Determine color: green if close > open (up), red if close < open (down)
            color = 'green' if close_price >= open_price else 'red'
            
            # Draw high-low line (wick)
            ax1.plot([x, x], [low_price, high_price], color='black', linewidth=0.5)
            
            # Draw open-close box (body)
            height = abs(close_price - open_price)
            bottom = min(open_price, close_price)
            
            if height == 0:  # Doji - draw a horizontal line
                ax1.plot([x-0.3, x+0.3], [close_price, close_price], color=color, linewidth=1.5)
            else:
                rect = plt.Rectangle((x-0.3, bottom), 0.6, height, 
                                    facecolor=color, edgecolor='black', linewidth=0.5, alpha=0.8)
                ax1.add_patch(rect)
    else:
        # Use line chart for longer periods (cleaner visualization)
        x_values = plot_df.index.values
        ax1.plot(x_values, plot_df['close'], linewidth=1.5, color='black', 
                label='Close Price', alpha=0.8)
        
        # Optionally show high/low range as shaded area
        ax1.fill_between(x_values, plot_df['low'], plot_df['high'], 
                        alpha=0.1, color='gray', label='High-Low Range')
    
    # Overlay Bollinger Bands
    if all(col in plot_df.columns for col in ['upper_band', 'middle_band', 'lower_band']):
        x_values = plot_df.index.values
        
        # Fill between upper and lower bands (the "stretched zone")
        ax1.fill_between(x_values, plot_df['lower_band'], plot_df['upper_band'], 
                         alpha=0.15, label='BB Stretched Zone', color='blue')
        
        # Plot band lines
        ax1.plot(x_values, plot_df['upper_band'], linewidth=1.5, alpha=0.7, 
                color='blue', linestyle='--', label='Upper Band ("Overpriced")')
        ax1.plot(x_values, plot_df['middle_band'], linewidth=2, color='orange', 
                label='Middle Band (Fair Value)')
        ax1.plot(x_values, plot_df['lower_band'], linewidth=1.5, alpha=0.7, 
                color='blue', linestyle='--', label='Lower Band ("On Sale")')
    
    ax1.set_ylabel('Price', fontsize=11, fontweight='bold')
    title_text = f'Panel 1: "The Action" - Price + Bollinger Bands - {title_suffix}' if title_suffix else 'Panel 1: "The Action" - Price + Bollinger Bands'
    ax1.set_title(title_text, fontsize=12, fontweight='bold', pad=10)
    ax1.legend(loc='upper left', fontsize=9, framealpha=0.9)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-0.5, len(plot_df) - 0.5)
    
    # ============================================================================
    # PANEL 2: "REGIME" - ADX + Directional Movement
    # ============================================================================
    
    if all(col in plot_df.columns for col in ['adx_value', 'di_plus', 'di_minus']):
        x_values = plot_df.index.values
        
        ax2.plot(x_values, plot_df['adx_value'], label='ADX (Trend Strength)', 
                linewidth=2, color='black')
        ax2.plot(x_values, plot_df['di_plus'], label='DI+ (Buyers)', 
                linewidth=1.5, alpha=0.8, color='green')
        ax2.plot(x_values, plot_df['di_minus'], label='DI- (Sellers)', 
                linewidth=1.5, alpha=0.8, color='red')
        
        # Add regime threshold lines
        ax2.axhline(y=25, color='gray', linestyle='--', alpha=0.6, 
                   label='Strong Trend (>25)', linewidth=1)
        ax2.axhline(y=20, color='gray', linestyle=':', alpha=0.6, 
                   label='Ranging Market (<20)', linewidth=1)
        
        ax2.set_ylabel('ADX Value', fontsize=10, fontweight='bold')
        title_text = f'Panel 2: "The Regime" - ADX Market Analyst - {title_suffix}' if title_suffix else 'Panel 2: "The Regime" - ADX Market Analyst'
        ax2.set_title(title_text, fontsize=11, fontweight='bold', pad=8)
        ax2.legend(loc='upper left', fontsize=8, framealpha=0.9, ncol=2)
        ax2.grid(True, alpha=0.3)
        ax2.set_xlim(-0.5, len(plot_df) - 0.5)
    
    # ============================================================================
    # PANEL 3: "CONFIRMATION" - Volume Conviction
    # ============================================================================
    
    if 'conviction_oscillator' in plot_df.columns:
        x_values = plot_df.index.values
        
        # Color bars based on sign (green = bullish, red = bearish)
        colors = ['green' if x > 0 else 'red' for x in plot_df['conviction_oscillator']]
        ax3.bar(x_values, plot_df['conviction_oscillator'], color=colors, alpha=0.6, width=0.8)
        ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
        
        ax3.set_ylabel('Conviction', fontsize=10, fontweight='bold')
        title_text = f'Panel 3: "The Confirmation" - Volume Analyst - {title_suffix}' if title_suffix else 'Panel 3: "The Confirmation" - Volume Analyst'
        ax3.set_title(title_text, fontsize=11, fontweight='bold', pad=8)
        ax3.grid(True, alpha=0.3)
        ax3.set_xlim(-0.5, len(plot_df) - 0.5)
    
    # ============================================================================
    # X-AXIS FORMATTING (Adaptive based on data density)
    # ============================================================================
    
    total_points = len(plot_df)
    
    # Calculate optimal tick spacing based on total points
    if total_points <= 20:
        tick_spacing = 1
        date_fmt = '%m/%d\n%H:%M'
    elif total_points <= 50:
        tick_spacing = max(3, total_points // 15)
        date_fmt = '%m/%d\n%H:%M'
    elif total_points <= 150:
        tick_spacing = max(8, total_points // 14)
        date_fmt = '%m/%d\n%H:%M'
    elif total_points <= 365:
        tick_spacing = max(15, total_points // 20)
        date_fmt = '%Y-%m-%d'  # Daily format
    else:
        tick_spacing = max(20, total_points // 25)
        date_fmt = '%Y-%m-%d'  # Weekly/Monthly format
    
    # Generate tick positions
    tick_positions = list(range(0, total_points, tick_spacing))
    if tick_positions[-1] != total_points - 1:
        tick_positions.append(total_points - 1)
    
    # Generate datetime labels
    tick_labels = []
    for pos in tick_positions:
        dt = plot_df['datetime'].iloc[pos]
        label = dt.strftime(date_fmt)
        tick_labels.append(label)
    
    # Apply to all axes
    for ax in [ax1, ax2, ax3]:
        ax.set_xticks(tick_positions)
        ax.set_xticklabels(tick_labels, fontsize=8, ha='center', rotation=45 if total_points > 150 else 0)
        
        # Only bottom panel gets x-label
        if ax == ax3:
            ax.set_xlabel('Trading Date & Time', fontsize=11, fontweight='bold', labelpad=8)
    
    # Add overall title with resolution info
    resolution_text = f'({strategy_desc})'
    fig.suptitle(f'Trinity Strategy Visualization {resolution_text}', 
                 fontsize=14, fontweight='bold', y=0.995)
    
    # Save figure
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig)


def plot_distributions(df: pd.DataFrame, fields: List[str], output_path: str, title_suffix: str = ''):
    """Plot distributions for Scout fields"""
    if df.empty:
        print('⚠ No data available for plotting distributions')
        return

    # Filter to only Scout indicator fields (not header fields)
    scout_fields = ['adx_value', 'di_plus', 'di_minus', 'upper_band', 'middle_band', 
                    'lower_band', 'conviction_oscillator']
    plot_fields = [f for f in scout_fields if f in df.columns and pd.api.types.is_numeric_dtype(df[f])]

    if not plot_fields:
        print('⚠ No Scout fields available for distribution plotting')
        return

    n_plots = len(plot_fields)
    n_cols = 3
    n_rows = (n_plots + 2) // 3

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
    axes = axes.flatten() if n_plots > 1 else [axes]

    for idx, field in enumerate(plot_fields):
        data = df[field].dropna()
        axes[idx].hist(data, bins=30, alpha=0.7, edgecolor='black', color='steelblue')
        axes[idx].set_xlabel(field, fontsize=9)
        axes[idx].set_ylabel('Frequency', fontsize=9)
        axes[idx].set_title(f'{field}', fontsize=10, fontweight='bold')
        axes[idx].grid(True, alpha=0.3)

        mean_val = data.mean()
        axes[idx].axvline(mean_val, color='red', linestyle='--', linewidth=2, 
                         label=f'Mean: {mean_val:.4f}')
        axes[idx].legend(fontsize=8)

    # Hide unused subplots
    for idx in range(len(plot_fields), len(axes)):
        axes[idx].axis('off')

    # Add overall title
    fig.suptitle(f'Distribution Analysis - {title_suffix}' if title_suffix else 'Distribution Analysis', 
                 fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close(fig)


def plot_correlation_matrix(df: pd.DataFrame, fields: List[str], output_path: str, title_suffix: str = ''):
    """Plot correlation matrix for Scout fields"""
    if df.empty:
        print('⚠ No data available for correlation analysis')
        return

    # Filter to only Scout indicator fields
    scout_fields = ['adx_value', 'di_plus', 'di_minus', 'upper_band', 'middle_band', 
                    'lower_band', 'conviction_oscillator']
    numeric_fields = [f for f in scout_fields if f in df.columns and pd.api.types.is_numeric_dtype(df[f])]

    if len(numeric_fields) < 2:
        print('⚠ Need at least 2 Scout fields for correlation analysis')
        return

    corr_df = df[numeric_fields].corr()

    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_df, annot=True, fmt='.3f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={'shrink': 0.8})
    
    title_text = f'Scout Correlation Matrix - {title_suffix}' if title_suffix else 'Scout Correlation Matrix'
    plt.title(title_text, fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(output_path, dpi=150, bbox_inches='tight')
    plt.close()


print('✓ All plotting functions defined (plot_trinity_scouts, plot_distributions, plot_correlation_matrix)')

## Automated Multi-Instrument Visualization

Connects to server and generates all charts for every instrument configured in `uout.json`.

**Process:**
1. Parse instruments from `uout.json` → `[('DCE', 'i<00>'), ('SHFE', 'cu<00>'), ('SHFE', 'sc<00>')]`
2. Connect once to server (efficient Pattern 2 from wos/10-visualization.md)
3. For each instrument:
   - Fetch data (reusing connection)
   - Generate 3 chart types → save to `tier-1_output/charts/<instrument>/`
4. Close connection

**Output Structure:**
```
tier-1_output/charts/
├── i/       (Iron Ore - DCE)
│   ├── trinity_scouts_DATES.png
│   ├── trinity_distributions_DATES.png
│   └── trinity_correlation_DATES.png
├── cu/      (Copper - SHFE)
│   ├── trinity_scouts_DATES.png
│   ├── trinity_distributions_DATES.png
│   └── trinity_correlation_DATES.png
└── sc/      (Crude Oil - SHFE)
    ├── trinity_scouts_DATES.png
    ├── trinity_distributions_DATES.png
    └── trinity_correlation_DATES.png
```

In [ ]:
# Parse instruments from uout.json
instruments = parse_uout_instruments()
print(f'📊 Found {len(instruments)} instruments to visualize:')
for market, code in instruments:
    print(f'   - {market}/{code}')

# Create output directory structure
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f'\n📁 Output directory: {OUTPUT_DIR}')

# Initialize fetcher and connect once (for indicators)
fetcher = TrinityStrategyDataFetcher(TOKEN, START_DATE, END_DATE)
await fetcher.connect()

# Process each instrument
print(f'\n{"="*70}')
print('Starting automated visualization for all instruments...')
print(f'{"="*70}\n')

for market, code in instruments:
    print(f'{"="*70}')
    print(f'📈 Processing {market}/{code}...')
    print(f'{"="*70}')
    
    # Fetch indicator data (private namespace - reuses fetcher connection)
    indicator_df = await fetcher.fetch(market, code)
    
    if indicator_df.empty:
        print(f'⚠ No indicator data returned for {market}/{code}\n')
        continue
    
    # Fetch price data (global namespace - SEPARATE connection)
    # ✅ FIX: Use standalone function with its own connection
    price_df = await fetch_price_data_standalone(TOKEN, START_DATE, END_DATE, market, code)
    
    if price_df.empty:
        print(f'⚠ No price data returned for {market}/{code}\n')
        continue
    
    # Merge price and indicator data on time_tag
    print(f'🔗 Merging price and indicator data...')
    merged_df = pd.merge(
        price_df, 
        indicator_df,
        on='time_tag',
        how='inner',  # Only keep timestamps where both exist
        suffixes=('_price', '_indicator')
    )
    
    # Use datetime from price data (they should be identical)
    if 'datetime_price' in merged_df.columns:
        merged_df['datetime'] = merged_df['datetime_price']
        merged_df.drop(['datetime_price', 'datetime_indicator'], axis=1, inplace=True, errors='ignore')
    
    # Reset index for continuous plotting
    merged_df = merged_df.reset_index(drop=True)
    
    print(f'✓ Merged dataset: {len(merged_df)} data points')
    print(f'  Date range: {merged_df["datetime"].min()} to {merged_df["datetime"].max()}')
    
    # Check for required fields
    required_ohlcv = ['open', 'high', 'low', 'close', 'volume']
    required_indicators = ['upper_band', 'middle_band', 'lower_band', 'adx_value', 'conviction_oscillator']
    
    missing_fields = [f for f in required_ohlcv + required_indicators if f not in merged_df.columns]
    if missing_fields:
        print(f'⚠ Missing required fields: {", ".join(missing_fields)}')
        print(f'  Skipping {market}/{code}\n')
        continue
    
    # Update fetcher's dataframe for distribution/correlation plots
    fetcher.df = merged_df
    
    # Create instrument-specific subdirectory
    instrument_name = code.replace('<00>', '')  # 'i<00>' -> 'i'
    chart_dir = os.path.join(OUTPUT_DIR, instrument_name)
    os.makedirs(chart_dir, exist_ok=True)
    
    # Generate all three chart types
    title_suffix = f'{market}/{code}'
    
    # 1. Scout time series chart WITH PRICE CANDLESTICKS (NEW!)
    scouts_file = os.path.join(chart_dir, f'trinity_scouts_{START_DATE}_{END_DATE}.png')
    plot_trinity_scouts(merged_df, scouts_file, title_suffix)
    print(f'✓ Saved: {scouts_file}')
    
    # 2. Distribution analysis chart
    dist_file = os.path.join(chart_dir, f'trinity_distributions_{START_DATE}_{END_DATE}.png')
    plot_distributions(merged_df, fetcher.available_fields, dist_file, title_suffix)
    print(f'✓ Saved: {dist_file}')
    
    # 3. Correlation matrix chart
    corr_file = os.path.join(chart_dir, f'trinity_correlation_{START_DATE}_{END_DATE}.png')
    plot_correlation_matrix(merged_df, fetcher.available_fields, corr_file, title_suffix)
    print(f'✓ Saved: {corr_file}')
    
    print()

# Close connection (cleanup)
await fetcher.close()

print(f'{"="*70}')
print('✅ All visualizations complete!')
print(f'{"="*70}')
print(f'\n📁 Charts saved to: {OUTPUT_DIR}/')
print(f'   Directory structure:')
for market, code in instruments:
    instrument_name = code.replace('<00>', '')
    print(f'   ├── {instrument_name}/')
    print(f'   │   ├── trinity_scouts_{START_DATE}_{END_DATE}.png')
    print(f'   │   ├── trinity_distributions_{START_DATE}_{END_DATE}.png')
    print(f'   │   └── trinity_correlation_{START_DATE}_{END_DATE}.png')